# Experimentation scripts for Harness evaluation (baseline)

## Global parameters

In [1]:
shell_log=False
hostAddress="10-0-0-33.nip.io"
from tqdm import tqdm, tqdm_notebook

### Mongo db & collection initialization

In [2]:
from pymongo import MongoClient
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import seaborn as sns

client = MongoClient("manager-mongo-manager")

db = client["private-recsys"]
collection_experiment = db["experiments"]



### Low level scripting functions

In [3]:
import subprocess
import shlex
import datetime
from dateutil.tz import tzlocal
import pandas as pd
import glob
import time
from time import sleep 
def run_command_async(command):
    if (isinstance(command, list) == False):
        commands = [command]
    else:
        commands=command
    processes = [None] * len(commands)
    for i in range(len(commands)):
        processes[i] = subprocess.Popen(shlex.split(commands[i]), stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    return processes

def wait_for_command_async(processes):
    while processes:
        for i, process in enumerate(processes):
            output = process.stdout.readline().decode()
            if output == '' and process.poll() is not None:
                processes.remove(process)
                break
            if output:
                now=datetime.datetime.now(tzlocal())
                strnow = now.strftime("%Y-%m-%d %H:%M:%S")
                print ("Log {0} - {1} : ".format(i,strnow) + output.strip())
    
    rc = process.poll()
    return rc    
def run_command(command, shell=False, log=True):
    print(command)
    if (isinstance(command, list) == False):
        commands = [command]
    else:
        commands=command
    processes = [None] * len(commands)
    for i in range(len(commands)):
        if log == True:
            print (commands[i])
        processes[i] = subprocess.Popen(shlex.split(commands[i]), stdout=subprocess.PIPE, stderr=subprocess.STDOUT, shell=shell)
    while processes:
        for i, process in enumerate(processes):
            output = process.stdout.readline().decode()
            if output == '' and process.poll() is not None:
                processes.remove(process)
                break
            if output and log:
                now=datetime.datetime.now(tzlocal())
                strnow = now.strftime("%Y-%m-%d %H:%M:%S")
                print ("Log {0} - {1} : ".format(i,strnow) + output.strip())
    
    rc = process.poll()
    return rc
import os

class cd:
    """Context manager for changing the current working directory"""
    def __init__(self, newPath):
        self.newPath = os.path.expanduser(newPath)

    def __enter__(self):
        self.savedPath = os.getcwd()
        os.chdir(self.newPath)

    def __exit__(self, etype, value, traceback):
        os.chdir(self.savedPath)
def get_strnow():
    now=datetime.datetime.now(tzlocal())
    return now.strftime("%Y%m%d%H%M%S")


### Module scope functions

In [4]:
def wait_for_http_status(url, code, duration):
    import requests
    import time
    print
    req_code = None
    start_time = time.time()
    end_time= time.time()
    while req_code != code and (end_time - start_time) < duration :
        try:
            time.sleep(1)
            r = requests.head(url)
            req_code = r.status_code
            print("\rWaiting status {} for {} seconds            ".format(code, end_time - start_time), end="\r", flush=True)
            return 0
        except requests.ConnectionError:
            print("failed to connect")
            return 1
        end_time = time.time()    

def wait_for_job(name, duration):
    with cd(".."):
        command = "./wait_for_job.sh {} {}".format(name, duration)
        result = run_command(command, log = shell_log)
        return result
        
def install_recsys(params):
    
    run_command("helm init --client-only", log = shell_log)
    #launch commands manually
    #with cd("PrivateRecSys/charts/private-recsys"):
    #   run_command("helm repo add main https://kubernetes-charts.storage.googleapis.com/")
    #    run_command("helm repo add elastic https://helm.elastic.co")
    #    run_command("helm dep update") # can't work https://github.com/helm/helm/issues/2998 : it should be done on the node
        
    with cd("../../charts/private-recsys"):
        run_command("kubectl apply -f ..", log = shell_log)
        param_str = ""
        for k in params.keys():
            param_str += "--set {}={} ".format(str(k).replace("!","."), str(params[k]).replace("!","."))
        command = "helm install --name recsys --namespace default . {} --wait --timeout {} ".format(param_str, params["timeout"])
        result = run_command(command, log = shell_log)
        return result    
    
def uninstall_recsys():
    with cd("../../charts/private-recsys"):
        result = run_command("helm delete --purge recsys", log = shell_log)
        if result == 0:
            sleep(120)
    return result

def install_stub(params):
    
    run_command("helm init --client-only")
    #launch commands manually
    #with cd("PrivateRecSys/charts/private-recsys"):
    #   run_command("helm repo add main https://kubernetes-charts.storage.googleapis.com/")
    #    run_command("helm repo add elastic https://helm.elastic.co")
    #    run_command("helm dep update") # can't work https://github.com/helm/helm/issues/2998 : it should be done on the node
        
    with cd("../../charts/stub-recsys"):
        param_str = ""
        for k in params.keys():
            param_str += "--set {}={} ".format(str(k).replace("!","."), str(params[k]).replace("!","."))
        command = "helm install --name stub --namespace default . {} --wait --timeout {} ".format(param_str, params["timeout"])
        result = run_command(command, log = shell_log)
        return result    

def uninstall_stub():
    with cd("../../charts/stub-recsys"):
        result = run_command("helm delete --purge stub", log = shell_log)
        if result == 0:
            sleep(30)
    return result

def launch_injector(params):
    with cd("../../charts/injector"):
        param_str = ""
        for k in params.keys():
            param_str += "--set {}={} ".format(str(k).replace("!","."), str(params[k]).replace("!","."))
        command = "helm install --name injector --namespace default . {} --wait --timeout {}".format(param_str, params["timeout"])
        result = run_command(command, log = shell_log)
    
    if result == 0:
        #result = run_command("kubectl wait --namespace default --for=condition=complete --timeout={}s job/injector-injector".format(params["timeout"]+600), log = shell_log)
        result = wait_for_job("injector-injector", params["timeout"]+600)
    return result
    
def uninstall_injector():
    with cd("../../charts/injector"):
        result = run_command("helm delete --purge injector", log = shell_log)
        if result == 0:
            sleep(30)
    return result


### Test scope functions

In [5]:
def is_harness_result_ok(collection):
    row = collection.find_one({"http_status":201})
    if row is not None:
        #print(row["content"]["result"])
        if len(row["content"]["result"]) > 0:
            return True
    else:
        print("Empty db")
    return False
def deploy_test(params):
    # uninstall previous launches
    print(uninstall_stub())
    print(uninstall_injector())
    print(uninstall_recsys())

    # init naming for experiment
    strnow = get_strnow()
    params_recsys = params["params_recsys"]
    params_injector = params["params_injector"]
    
    params["name"] = params["name"] + strnow
    params_injector["name"] = params_injector["name"] + strnow
    params_recsys["name"] = params_recsys["name"] + strnow
    
    # add labels with names
    params_injector["experiment"] = params["name"] 
    params_recsys["experiment"] = params["name"] 
    params_recsys["proxy-sgx!experiment"] = params["name"]
    params_recsys["harness!experiment"] = params["name"]
    
    mongo_id = collection_experiment.insert_one(params).inserted_id
    collection_experiment.update_one({'_id':mongo_id}, {"$set": params}, upsert=False)

    # init step recsys
    params_recsys["start_time"] = time.time()
    params["status"] = "recsys"
    collection_experiment.update_one({'_id':mongo_id}, {"$set": params}, upsert=False)
    
    cpt = 0
    err = None
    while (err is None or err != 0) and cpt < 5:
        
        if params["stub"] != "1":
            err=install_recsys(params_recsys)
        else:
            err=install_stub(params_recsys)
        print("err={}".format(err))
        if err != 0:
            if params["stub"] != "1":
                uninstall_recsys()
            else:
                uninstall_stub()
        cpt += 1

    params_recsys["end_time"] = time.time()
    params["status"] = "recsys"
    if err != 0:
        params["status"] = "recsys_error"

    collection_experiment.update_one({'_id':mongo_id}, {"$set": params}, upsert=False)
    return params["status"]    
def run_test(params):
    # uninstall previous launches
    print(uninstall_injector())

    # init naming for experiment
    strnow = get_strnow()
    params_recsys = params["params_recsys"]
    params_injector = params["params_injector"]
    
    params["name"] = params["name"] + strnow
    params_injector["name"] = params_injector["name"] + strnow
    params_recsys["name"] = params_recsys["name"] + strnow
    
    # add labels with names
    params_injector["experiment"] = params["name"] 
    params_recsys["experiment"] = params["name"] 
    params_recsys["proxy-sgx!experiment"] = params["name"]
    params_recsys["harness!experiment"] = params["name"]
    
    mongo_id = collection_experiment.insert_one(params).inserted_id
    collection_experiment.update_one({'_id':mongo_id}, {"$set": params}, upsert=False)
   
    params_injector["start_time"] = time.time()
    params["status"] = "injector"
    collection_experiment.update_one({'_id':mongo_id}, {"$set": params}, upsert=False)
    if params["stub"] != "1":
        time.sleep(120)
    cpt = 0

    err = launch_injector(params_injector)
    print("Injector deployment err : {}".format(err))
    #uninstall_injector()

    if err != 0:
        if err == 1:
            # failed injector, we assume it is harness
            params["status"] = "harness_error"
        return params["status"]
    
    params_injector["end_time"] = time.time()
    params["status"] = "injector"
    collection_experiment.update_one({'_id':mongo_id}, {"$set": params}, upsert=False)

    params_injector["error"] = err
    params_injector["end_time"] = time.time()
    #now=datetime.datetime.now(tzlocal())
    #strnow = now.strftime("%Y%m%d%H%M%S")



    coll_name = "private-recsys.{}.locust_request_result".format(params_injector["name"]) 
    coll = db[coll_name]
    params["status"] = "finished"
    
    #if params["stub"] != "1":
    #    if is_harness_result_ok(coll) == False:
    #        params["status"] = "harness_error"
    #print("{} - {}".format(coll_name, is_harness_result_ok(coll)))
        
    params["end_time"] = time.time()
    collection_experiment.update_one({'_id':mongo_id}, {"$set": params}, upsert=False)
    return params["status"]


### Test functions

In [6]:
import itertools
import collections
def unpack_dict(target):
    ar = collections.OrderedDict()
    for k in target.keys():
        if isinstance(target[k], list):
            ar[k] = target[k]
    print(ar.keys())
    combinations = [dict(zip(ar.keys(), i)) for i in itertools.product(*ar.values())]
    result = []
    for c in combinations:
        print(c)
        d = target.copy()
        d.update(c)
        result.append(d)
    return result, ar

def rename_db(old, new):
    import pymongo
    from pymongo import MongoClient
    client = MongoClient("manager-mongo-manager")
    client.admin.command('copydb',
                         fromdb=old,
                         todb=new)
    client.drop_database(old)
    
def loop_run_test(params_packed):
    list_params_recsys, arrays_params_recsys = unpack_dict(params_packed["params_recsys"])
    list_params_injector, arrays_params_injector = unpack_dict(params_packed["params_injector"])

    strnow = get_strnow()
    print("db:" + "-".join(list(arrays_params_recsys))+"_"+"-".join(list(arrays_params_injector))+"_"+strnow)

    print(list_params_injector)
    first=True
    for params_recsys in list_params_recsys:
        params = params_packed.copy()
        params["params_recsys"] = params_recsys.copy()
        result = deploy_test(params.copy())
        
        if result != "recsys":
            break
        for params_injector in list_params_injector:
            for i in range(5):
                params = params_packed.copy()
                params["params_recsys"] = params_recsys.copy()
                params["params_injector"] = params_injector.copy()
                if first == True:
                    print("First launch")
                    
                    result = run_test(params.copy())
                    if result != "finished":
                        print("Launch {} in error, redeploying recsys.".format(i))
                        result = deploy_test(params.copy())
                        
                    else:
                        first=False
                        break

                else:
                    print("Not first, no init/train needed")
                    params["params_injector"]["harness"] = False

                    result = run_test(params.copy())
                    if result != "harness_error":
                        break
    #rename database 
    #rename_db("private-recsys", "-".join(list(arrays_params_recsys))+"_"+"-".join(list(arrays_params_injector))+"_"+strnow)

def fusion_experiments_db(name_from, name_to, drop = False):
    import pymongo
    from pymongo import MongoClient
    client = MongoClient("manager-mongo-manager")    
    if drop == True:
        client.drop_database(name_to)
    from pymongo import MongoClient
    client = MongoClient("manager-mongo-manager")
    database_names = client.list_database_names()
    for database_name in database_names:
        if name_from in database_name:
            print(database_name)
            elements = list(client[database_name]["experiments"].find())
            client[name_to]["experiments"].insert_many(elements) 
            for coll_name in client[database_name].list_collection_names(nameOnly=True):
                if coll_name != "experiments":
                    elements = list(client[database_name][coll_name].find())
                    client[name_to][coll_name].insert_many(elements)
    
def remove_working_db():
    from pymongo import MongoClient
    client = MongoClient("manager-mongo-manager")   
    client.drop_database("private-recsys")


## Remove deployment

In [ ]:
# reset 

print(uninstall_stub())
print(uninstall_injector())
print(uninstall_recsys())

In [ ]:
fusion_experiments_db("private-recsys", "debug_harness_8e", True)   

# macro1a - performances harness

In [ ]:
remove_working_db()
for replicas in tqdm_notebook([1], "replicas"):
    for client in tqdm_notebook(["1","25","50","75","100"], "input rps"):
        for identifier in tqdm_notebook(range(6), "runs per config"):
            concurrency = 10

            name = "macro1a_{}_{}".format(client, identifier)
            params_recsys = {
                "hostAddress":hostAddress,
                "mongo!replicaSet!replicas!secondary":0,
                "elastic!replicas":3,
                "harness!replicaCount":1,
                #"harness!image!tag": "develop-2020-01-28",
                "proxy-sgx!replicaCount":1,
                "proxy-sgx!proxyType1":"1",
                "proxy-sgx!proxyType2":"2",
                "proxy-sgx!enableEncryption":"1",
                "mongo!replicaSet!enabled": "false",
                "elastic!esJavaOpts":'"-Xmx16g -Xms16g"',
                "elastic!resources!requests!cpu":"1500m",
                "elastic!resources!requests!memory":"17Gi",
                "elastic!resources!limits!cpu":"2000m",
                "elastic!resources!limits!memory":"17Gi",
                "name":name,
                "timeout":450,
            #    "mongo!clusterDomain":"nuc.local"  # Louvain
                "mongo!clusterDomain":"cluster.local" # Bordeaux
            }
            params_injector={
                "duration":300,
                "clients":client,
                "replicaCount":1,
                "targetHost":"http://recsys-harness:9090",       
                "concurrency":[concurrency],
                "harnessReplicas": replicas,
                "timeout":1200,
                "asyncFluentd":"false",
                "timeoutRequest":800,
                "periodRequest":1000,
                "enableEncryption":"0",
                "randomShift":"1",
                "name":name,
                "harness":"true" # false if stub
            }
            params = {
                "start_time":time.time(),
                "params_recsys":params_recsys,
                "params_injector":params_injector,
                "name:":name,
                "error_module":"",
                "error":0,
                "status":"created",
                "name":name,
                "stub":"0"
            }

            loop_run_test(params)
fusion_experiments_db("private-recsys", "macro1a", False)   

# macro 1b : 3 harnesses (250)

In [ ]:
remove_working_db()
for replicas in tqdm_notebook([3], "replicas"):
    for client in tqdm_notebook(["50","100","150","200","250"], "input rps"):
        for identifier in tqdm_notebook(range(6), "runs per config"):
            concurrency = 10

            name = "macro1b_{}_{}".format(client, identifier)
            params_recsys = {
                "hostAddress":hostAddress,
                "mongo!replicaSet!replicas!secondary":0,
                "elastic!replicas":3,
                "harness!replicaCount":1,
                #"harness!image!tag": "develop-2020-01-28",
                "proxy-sgx!replicaCount":1,
                "proxy-sgx!proxyType1":"1",
                "proxy-sgx!proxyType2":"2",
                "proxy-sgx!enableEncryption":"1",
                "mongo!replicaSet!enabled": "false",
                "elastic!esJavaOpts":'"-Xmx16g -Xms16g"',
                "elastic!resources!requests!cpu":"1500m",
                "elastic!resources!requests!memory":"17Gi",
                "elastic!resources!limits!cpu":"2000m",
                "elastic!resources!limits!memory":"17Gi",
                "name":name,
                "timeout":450,
            #    "mongo!clusterDomain":"nuc.local"  # Louvain
                "mongo!clusterDomain":"cluster.local" # Bordeaux
            }
            params_injector={
                "duration":300,
                "clients":client,
                "replicaCount":1,
                "targetHost":"http://recsys-harness:9090",       
                "concurrency":[concurrency],
                "harnessReplicas": replicas,
                "timeout":1200,
                "asyncFluentd":"false",
                "timeoutRequest":800,
                "periodRequest":1000,
                "enableEncryption":"0",
                "randomShift":"1",
                "name":name,
                "harness":"true" # false if stub
            }
            params = {
                "start_time":time.time(),
                "params_recsys":params_recsys,
                "params_injector":params_injector,
                "name:":name,
                "error_module":"",
                "error":0,
                "status":"created",
                "name":name,
                "stub":"0"
            }

            loop_run_test(params)
fusion_experiments_db("private-recsys", "macro1b", False)   

# macro 1c : 6 harness

In [ ]:
remove_working_db()
for replicas in tqdm_notebook([6], "replicas"):
    for client in tqdm_notebook(["50","250","500"], "input rps"):
        for identifier in tqdm_notebook(range(6), "runs per config"):
            concurrency = 10

            name = "macro1c_{}_{}".format(client, identifier)
            params_recsys = {
                "hostAddress":hostAddress,
                "mongo!replicaSet!replicas!secondary":0,
                "elastic!replicas":3,
                "harness!replicaCount":1,
                #"harness!image!tag": "develop-2020-01-28",
                "proxy-sgx!replicaCount":1,
                "proxy-sgx!proxyType1":"1",
                "proxy-sgx!proxyType2":"2",
                "proxy-sgx!enableEncryption":"1",
                "mongo!replicaSet!enabled": "false",
                "elastic!esJavaOpts":'"-Xmx16g -Xms16g"',
                "elastic!resources!requests!cpu":"1500m",
                "elastic!resources!requests!memory":"17Gi",
                "elastic!resources!limits!cpu":"2000m",
                "elastic!resources!limits!memory":"17Gi",
                "name":name,
                "timeout":450,
            #    "mongo!clusterDomain":"nuc.local"  # Louvain
                "mongo!clusterDomain":"cluster.local" # Bordeaux
            }
            params_injector={
                "duration":300,
                "clients":client,
                "replicaCount":1,
                "targetHost":"http://recsys-harness:9090",       
                "concurrency":[concurrency],
                "harnessReplicas": replicas,
                "timeout":1200,
                "asyncFluentd":"false",
                "timeoutRequest":800,
                "periodRequest":1000,
                "enableEncryption":"0",
                "randomShift":"1",
                "name":name,
                "harness":"true" # false if stub
            }
            params = {
                "start_time":time.time(),
                "params_recsys":params_recsys,
                "params_injector":params_injector,
                "name:":name,
                "error_module":"",
                "error":0,
                "status":"created",
                "name":name,
                "stub":"0"
            }

            loop_run_test(params)
fusion_experiments_db("private-recsys", "macro1c", False)   

# macro 1d : 9 Harness

In [ ]:
remove_working_db()
for replicas in tqdm_notebook([9], "replicas"):
    #for client in tqdm_notebook(["50","250","500","750"], "input rps"):
    #for client in tqdm_notebook(["750"], "input rps"):
    for client in tqdm_notebook(["50","250","500"], "input rps"):
        for identifier in tqdm_notebook(range(6), "runs per config"):
            concurrency = 10

            name = "macro1d_{}_{}".format(client, identifier)
            params_recsys = {
                "hostAddress":hostAddress,
                "mongo!replicaSet!replicas!secondary":0,
                "elastic!replicas":3,
                "harness!replicaCount":1,
                #"harness!image!tag": "develop-2020-01-28",
                "proxy-sgx!replicaCount":1,
                "proxy-sgx!proxyType1":"1",
                "proxy-sgx!proxyType2":"2",
                "proxy-sgx!enableEncryption":"1",
                "mongo!replicaSet!enabled": "false",
                "elastic!esJavaOpts":'"-Xmx16g -Xms16g"',
                "elastic!resources!requests!cpu":"1500m",
                "elastic!resources!requests!memory":"17Gi",
                "elastic!resources!limits!cpu":"2000m",
                "elastic!resources!limits!memory":"17Gi",
                "name":name,
                "timeout":450,
            #    "mongo!clusterDomain":"nuc.local"  # Louvain
                "mongo!clusterDomain":"cluster.local" # Bordeaux
            }
            params_injector={
                "duration":300,
                "clients":client,
                "replicaCount":1,
                "targetHost":"http://recsys-harness:9090",       
                "concurrency":[concurrency],
                "harnessReplicas": replicas,
                "timeout":1200,
                "asyncFluentd":"false",
                "timeoutRequest":800,
                "periodRequest":1000,
                "enableEncryption":"0",
                "randomShift":"1",
                "name":name,
                "harness":"true" # false if stub
            }
            params = {
                "start_time":time.time(),
                "params_recsys":params_recsys,
                "params_injector":params_injector,
                "name:":name,
                "error_module":"",
                "error":0,
                "status":"created",
                "name":name,
                "stub":"0"
            }

            loop_run_test(params)
fusion_experiments_db("private-recsys", "macro1d", False)   

In [8]:
remove_working_db()
for replicas in tqdm_notebook([9], "replicas"):
    for client in tqdm_notebook(["375"], "input rps"):
        for identifier in tqdm_notebook(range(6), "runs per config"):
            concurrency = 10

            name = "macro1d_{}_{}".format(client, identifier)
            params_recsys = {
                "hostAddress":hostAddress,
                "mongo!replicaSet!replicas!secondary":0,
                "elastic!replicas":3,
                "harness!replicaCount":1,
                #"harness!image!tag": "develop-2020-01-28",
                "proxy-sgx!replicaCount":1,
                "proxy-sgx!proxyType1":"1",
                "proxy-sgx!proxyType2":"2",
                "proxy-sgx!enableEncryption":"1",
                "mongo!replicaSet!enabled": "false",
                "elastic!esJavaOpts":'"-Xmx16g -Xms16g"',
                "elastic!resources!requests!cpu":"1500m",
                "elastic!resources!requests!memory":"17Gi",
                "elastic!resources!limits!cpu":"2000m",
                "elastic!resources!limits!memory":"17Gi",
                "name":name,
                "timeout":450,
            #    "mongo!clusterDomain":"nuc.local"  # Louvain
                "mongo!clusterDomain":"cluster.local" # Bordeaux
            }
            params_injector={
                "duration":300,
                "clients":client,
                "replicaCount":2,
                "targetHost":"http://recsys-harness:9090",       
                "concurrency":[concurrency],
                "harnessReplicas": replicas,
                "timeout":1200,
                "asyncFluentd":"false",
                "timeoutRequest":800,
                "periodRequest":1000,
                "enableEncryption":"0",
                "randomShift":"1",
                "name":name,
                "harness":"true" # false if stub
            }
            params = {
                "start_time":time.time(),
                "params_recsys":params_recsys,
                "params_injector":params_injector,
                "name:":name,
                "error_module":"",
                "error":0,
                "status":"created",
                "name":name,
                "stub":"0"
            }

            loop_run_test(params)
fusion_experiments_db("private-recsys", "macro1d_ext", True)   

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


odict_keys([])
{}
odict_keys(['concurrency'])
{'concurrency': 10}
db:_concurrency_20200429231532
[{'duration': 300, 'clients': '375', 'replicaCount': 2, 'targetHost': 'http://recsys-harness:9090', 'concurrency': 10, 'harnessReplicas': 9, 'timeout': 1200, 'asyncFluentd': 'false', 'timeoutRequest': 800, 'periodRequest': 1000, 'enableEncryption': '0', 'randomShift': '1', 'name': 'macro1d_375_0', 'harness': 'true'}]
helm delete --purge stub
1
helm delete --purge injector
0
helm delete --purge recsys
1
helm init --client-only
kubectl apply -f ..
helm install --name recsys --namespace default . --set hostAddress=10-0-0-33.nip.io --set mongo.replicaSet.replicas.secondary=0 --set elastic.replicas=3 --set harness.replicaCount=1 --set proxy-sgx.replicaCount=1 --set proxy-sgx.proxyType1=1 --set proxy-sgx.proxyType2=2 --set proxy-sgx.enableEncryption=1 --set mongo.replicaSet.enabled=false --set elastic.esJavaOpts="-Xmx16g -Xms16g" --set elastic.resources.requests.cpu=1500m --set elastic.resources.

# macro 1e : 12 harness - 2 injectors

In [ ]:
remove_working_db()
for replicas in tqdm_notebook([12], "replicas"):
    #for client in tqdm_notebook(["25","125","250","375","500"], "input rps"):
    for client in tqdm_notebook(["500"], "input rps"):
        for identifier in tqdm_notebook(range(6), "runs per config"):
            concurrency = 10

            name = "macro1e_{}_{}".format(client, identifier)
            params_recsys = {
                "hostAddress":hostAddress,
                "mongo!replicaSet!replicas!secondary":0,
                "elastic!replicas":3,
                "harness!replicaCount":1,
                #"harness!image!tag": "develop-2020-01-28",
                "proxy-sgx!replicaCount":1,
                "proxy-sgx!proxyType1":"1",
                "proxy-sgx!proxyType2":"2",
                "proxy-sgx!enableEncryption":"1",
                "mongo!replicaSet!enabled": "false",
                "elastic!esJavaOpts":'"-Xmx16g -Xms16g"',
                "elastic!resources!requests!cpu":"1500m",
                "elastic!resources!requests!memory":"17Gi",
                "elastic!resources!limits!cpu":"2000m",
                "elastic!resources!limits!memory":"17Gi",
                "name":name,
                "timeout":450,
            #    "mongo!clusterDomain":"nuc.local"  # Louvain
                "mongo!clusterDomain":"cluster.local" # Bordeaux
            }
            params_injector={
                "duration":300,
                "clients":client,
                "replicaCount":2,
                "targetHost":"http://recsys-harness:9090",       
                "concurrency":[concurrency],
                "harnessReplicas": replicas,
                "timeout":1200,
                "asyncFluentd":"false",
                "timeoutRequest":800,
                "periodRequest":1000,
                "enableEncryption":"0",
                "randomShift":"1",
                "name":name,
                "harness":"true" # false if stub
            }
            params = {
                "start_time":time.time(),
                "params_recsys":params_recsys,
                "params_injector":params_injector,
                "name:":name,
                "error_module":"",
                "error":0,
                "status":"created",
                "name":name,
                "stub":"0"
            }

            loop_run_test(params)
fusion_experiments_db("private-recsys", "macro1e", False)   

# test macro 2e : 12h, 4x2 pprox

In [ ]:
remove_working_db()
for replicas in tqdm_notebook([12], "replicas"):
    for client in tqdm_notebook(["25","125","250","375","500"], "input rps"):
        for identifier in tqdm_notebook(range(5), "runs per config"):
            concurrency = 10

            name = "macro2e_{}_{}".format(client, identifier)
            params_recsys = {
                "hostAddress":hostAddress,
                "mongo!replicaSet!replicas!secondary":0,
                "elastic!replicas":3,
                "harness!replicaCount":1,
                "proxy-sgx!replicaCount":4,
                "proxy-sgx!proxyType1":"1",
                "proxy-sgx!proxyType2":"2",
                "proxy-sgx!enableEncryption":"1",
                "proxy-sgx!bufferShuffling":"10",                
                "proxy-sgx!debug":"0",                
                "mongo!replicaSet!enabled": "false",
                "elastic!esJavaOpts":'"-Xmx16g -Xms16g"',
                "elastic!resources!requests!cpu":"1500m",
                "elastic!resources!requests!memory":"17Gi",
                "elastic!resources!limits!cpu":"2000m",
                "elastic!resources!limits!memory":"17Gi",
                "name":name,
                "timeout":450,
            #    "mongo!clusterDomain":"nuc.local"  # Louvain
                "mongo!clusterDomain":"cluster.local" # Bordeaux
            }
            params_injector={
                "duration":300,
                "clients":client,
                "replicaCount":2,
                "targetHost":["http://recsys-proxy-sgx-1:8080"], # pprox                
                "concurrency":[concurrency],
                "harnessReplicas": replicas,
                "timeout":1200,
                "asyncFluentd":"false",
                "timeoutRequest":800,
                "periodRequest":1000,
                "enableEncryption":"1",
                "randomShift":"1",
                "name":name,
                "harness":"true" # false if stub
            }
            params = {
                "start_time":time.time(),
                "params_recsys":params_recsys,
                "params_injector":params_injector,
                "name:":name,
                "error_module":"",
                "error":0,
                "status":"created",
                "name":name,
                "stub":"0"
            }

            loop_run_test(params)
fusion_experiments_db("private-recsys", "macro2e", False)   

# test macro 2b : 3h, 1x2 pprox

In [17]:
remove_working_db()
for replicas in tqdm_notebook([3], "replicas"):
#    for client in tqdm_notebook(["50","100","150","200","250"], "input rps"):
#        for identifier in tqdm_notebook(range(6), "runs per config"):
    for client in tqdm_notebook(["200"], "input rps"):
        for identifier in tqdm_notebook(range(1), "runs per config"):
            concurrency = 10

            name = "macro2b_{}_{}".format(client, identifier)
            params_recsys = {
                "hostAddress":hostAddress,
                "mongo!replicaSet!replicas!secondary":0,
                "elastic!replicas":3,
                "harness!replicaCount":1,
                "proxy-sgx!replicaCount":1,
                "proxy-sgx!proxyType1":"1",
                "proxy-sgx!proxyType2":"2",
                "proxy-sgx!enableEncryption":"1",
                "proxy-sgx!bufferShuffling":"10", 
                "proxy-sgx!debug":"0",                    
                "mongo!replicaSet!enabled": "false",
                "elastic!esJavaOpts":'"-Xmx16g -Xms16g"',
                "elastic!resources!requests!cpu":"1500m",
                "elastic!resources!requests!memory":"17Gi",
                "elastic!resources!limits!cpu":"2000m",
                "elastic!resources!limits!memory":"17Gi",
                "name":name,
                "timeout":450,
            #    "mongo!clusterDomain":"nuc.local"  # Louvain
                "mongo!clusterDomain":"cluster.local" # Bordeaux
            }
            params_injector={
                "duration":300,
                "clients":client,
                "replicaCount":1,
                "targetHost":["http://recsys-proxy-sgx-1:8080"], # pprox                
                "concurrency":[concurrency],
                "harnessReplicas": replicas,
                "timeout":1200,
                "asyncFluentd":"false",
                "timeoutRequest":800,
                "periodRequest":1000,
                "enableEncryption":"1",
                "randomShift":"1",
                "name":name,
                "harness":"true" # false if stub
            }
            params = {
                "start_time":time.time(),
                "params_recsys":params_recsys,
                "params_injector":params_injector,
                "name:":name,
                "error_module":"",
                "error":0,
                "status":"created",
                "name":name,
                "stub":"0"
            }

            loop_run_test(params)
fusion_experiments_db("private-recsys", "macro2b", False)   

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


odict_keys([])
{}
odict_keys(['targetHost', 'concurrency'])
{'targetHost': 'http://recsys-proxy-sgx-1:8080', 'concurrency': 10}
db:_targetHost-concurrency_20200427065216
[{'duration': 300, 'clients': '200', 'replicaCount': 1, 'targetHost': 'http://recsys-proxy-sgx-1:8080', 'concurrency': 10, 'harnessReplicas': 3, 'timeout': 1200, 'asyncFluentd': 'false', 'timeoutRequest': 800, 'periodRequest': 1000, 'enableEncryption': '1', 'randomShift': '1', 'name': 'macro2b_200_0', 'harness': 'true'}]
helm delete --purge stub
1
helm delete --purge injector
0
helm delete --purge recsys
0
helm init --client-only
kubectl apply -f ..
helm install --name recsys --namespace default . --set hostAddress=10-0-0-33.nip.io --set mongo.replicaSet.replicas.secondary=0 --set elastic.replicas=3 --set harness.replicaCount=1 --set proxy-sgx.replicaCount=1 --set proxy-sgx.proxyType1=1 --set proxy-sgx.proxyType2=2 --set proxy-sgx.enableEncryption=1 --set proxy-sgx.bufferShuffling=10 --set proxy-sgx.debug=0 --set mongo

In [ ]:
fusion_experiments_db("private-recsys", "macro2b_before", False)   

# macro 2c

In [12]:
remove_working_db()
for replicas in tqdm_notebook([6], "replicas"):
    for client in tqdm_notebook(["50","250","500"], "input rps"):
        for identifier in tqdm_notebook(range(6), "runs per config"):
            concurrency = 10

            name = "macro2c_{}_{}".format(client, identifier)
            params_recsys = {
                "hostAddress":hostAddress,
                "mongo!replicaSet!replicas!secondary":0,
                "elastic!replicas":3,
                "harness!replicaCount":1,
                "proxy-sgx!replicaCount":2,
                "proxy-sgx!proxyType1":"1",
                "proxy-sgx!proxyType2":"2",
                "proxy-sgx!enableEncryption":"1",
                "proxy-sgx!bufferShuffling":"10", 
                "proxy-sgx!debug":"0",                    
                "mongo!replicaSet!enabled": "false",
                "elastic!esJavaOpts":'"-Xmx16g -Xms16g"',
                "elastic!resources!requests!cpu":"1500m",
                "elastic!resources!requests!memory":"17Gi",
                "elastic!resources!limits!cpu":"2000m",
                "elastic!resources!limits!memory":"17Gi",
                "name":name,
                "timeout":450,
            #    "mongo!clusterDomain":"nuc.local"  # Louvain
                "mongo!clusterDomain":"cluster.local" # Bordeaux
            }
            params_injector={
                "duration":300,
                "clients":client,
                "replicaCount":1,
                "targetHost":["http://recsys-proxy-sgx-1:8080"], # pprox                
                "concurrency":[concurrency],
                "harnessReplicas": replicas,
                "timeout":1200,
                "asyncFluentd":"false",
                "timeoutRequest":800,
                "periodRequest":1000,
                "enableEncryption":"1",
                "randomShift":"1",
                "name":name,
                "harness":"true" # false if stub
            }
            params = {
                "start_time":time.time(),
                "params_recsys":params_recsys,
                "params_injector":params_injector,
                "name:":name,
                "error_module":"",
                "error":0,
                "status":"created",
                "name":name,
                "stub":"0"
            }

            loop_run_test(params)
fusion_experiments_db("private-recsys", "macro2c", False)   

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


odict_keys([])
{}
odict_keys(['targetHost', 'concurrency'])
{'targetHost': 'http://recsys-proxy-sgx-1:8080', 'concurrency': 10}
db:_targetHost-concurrency_20200426024908
[{'duration': 300, 'clients': '50', 'replicaCount': 1, 'targetHost': 'http://recsys-proxy-sgx-1:8080', 'concurrency': 10, 'harnessReplicas': 6, 'timeout': 1200, 'asyncFluentd': 'false', 'timeoutRequest': 800, 'periodRequest': 1000, 'enableEncryption': '1', 'randomShift': '1', 'name': 'macro2c_50_0', 'harness': 'true'}]
helm delete --purge stub
1
helm delete --purge injector
1
helm delete --purge recsys
0
helm init --client-only
kubectl apply -f ..
helm install --name recsys --namespace default . --set hostAddress=10-0-0-33.nip.io --set mongo.replicaSet.replicas.secondary=0 --set elastic.replicas=3 --set harness.replicaCount=1 --set proxy-sgx.replicaCount=2 --set proxy-sgx.proxyType1=1 --set proxy-sgx.proxyType2=2 --set proxy-sgx.enableEncryption=1 --set proxy-sgx.bufferShuffling=10 --set proxy-sgx.debug=0 --set mongo.r

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


odict_keys([])
{}
odict_keys(['targetHost', 'concurrency'])
{'targetHost': 'http://recsys-proxy-sgx-1:8080', 'concurrency': 10}
db:_targetHost-concurrency_20200426050417
[{'duration': 300, 'clients': '250', 'replicaCount': 1, 'targetHost': 'http://recsys-proxy-sgx-1:8080', 'concurrency': 10, 'harnessReplicas': 6, 'timeout': 1200, 'asyncFluentd': 'false', 'timeoutRequest': 800, 'periodRequest': 1000, 'enableEncryption': '1', 'randomShift': '1', 'name': 'macro2c_250_0', 'harness': 'true'}]
helm delete --purge stub
1
helm delete --purge injector
0
helm delete --purge recsys
0
helm init --client-only
kubectl apply -f ..
helm install --name recsys --namespace default . --set hostAddress=10-0-0-33.nip.io --set mongo.replicaSet.replicas.secondary=0 --set elastic.replicas=3 --set harness.replicaCount=1 --set proxy-sgx.replicaCount=2 --set proxy-sgx.proxyType1=1 --set proxy-sgx.proxyType2=2 --set proxy-sgx.enableEncryption=1 --set proxy-sgx.bufferShuffling=10 --set proxy-sgx.debug=0 --set mongo

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


odict_keys([])
{}
odict_keys(['targetHost', 'concurrency'])
{'targetHost': 'http://recsys-proxy-sgx-1:8080', 'concurrency': 10}
db:_targetHost-concurrency_20200426072851
[{'duration': 300, 'clients': '500', 'replicaCount': 1, 'targetHost': 'http://recsys-proxy-sgx-1:8080', 'concurrency': 10, 'harnessReplicas': 6, 'timeout': 1200, 'asyncFluentd': 'false', 'timeoutRequest': 800, 'periodRequest': 1000, 'enableEncryption': '1', 'randomShift': '1', 'name': 'macro2c_500_0', 'harness': 'true'}]
helm delete --purge stub
1
helm delete --purge injector
0
helm delete --purge recsys
0
helm init --client-only
kubectl apply -f ..
helm install --name recsys --namespace default . --set hostAddress=10-0-0-33.nip.io --set mongo.replicaSet.replicas.secondary=0 --set elastic.replicas=3 --set harness.replicaCount=1 --set proxy-sgx.replicaCount=2 --set proxy-sgx.proxyType1=1 --set proxy-sgx.proxyType2=2 --set proxy-sgx.enableEncryption=1 --set proxy-sgx.bufferShuffling=10 --set proxy-sgx.debug=0 --set mongo

# macro 2d 0-500

In [16]:
remove_working_db()
for replicas in tqdm_notebook([9], "replicas"):
    for client in tqdm_notebook(["50","250","500"], "input rps"):
        for identifier in tqdm_notebook(range(6), "runs per config"):
            concurrency = 10

            name = "macro2d_{}_{}".format(client, identifier)
            params_recsys = {
                "hostAddress":hostAddress,
                "mongo!replicaSet!replicas!secondary":0,
                "elastic!replicas":3,
                "harness!replicaCount":1,
                "proxy-sgx!replicaCount":3,
                "proxy-sgx!proxyType1":"1",
                "proxy-sgx!proxyType2":"2",
                "proxy-sgx!enableEncryption":"1",
                "proxy-sgx!bufferShuffling":"10", 
                "proxy-sgx!debug":"0",                    
                "mongo!replicaSet!enabled": "false",
                "elastic!esJavaOpts":'"-Xmx16g -Xms16g"',
                "elastic!resources!requests!cpu":"1500m",
                "elastic!resources!requests!memory":"17Gi",
                "elastic!resources!limits!cpu":"2000m",
                "elastic!resources!limits!memory":"17Gi",
                "name":name,
                "timeout":450,
            #    "mongo!clusterDomain":"nuc.local"  # Louvain
                "mongo!clusterDomain":"cluster.local" # Bordeaux
            }
            params_injector={
                "duration":300,
                "clients":client,
                "replicaCount":2,
                "targetHost":["http://recsys-proxy-sgx-1:8080"], # pprox                
                "concurrency":[concurrency],
                "harnessReplicas": replicas,
                "timeout":1200,
                "asyncFluentd":"false",
                "timeoutRequest":800,
                "periodRequest":1000,
                "enableEncryption":"1",
                "randomShift":"1",
                "name":name,
                "harness":"true" # false if stub
            }
            params = {
                "start_time":time.time(),
                "params_recsys":params_recsys,
                "params_injector":params_injector,
                "name:":name,
                "error_module":"",
                "error":0,
                "status":"created",
                "name":name,
                "stub":"0"
            }

            loop_run_test(params)
fusion_experiments_db("private-recsys", "macro2d", False)   

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


odict_keys([])
{}
odict_keys(['targetHost', 'concurrency'])
{'targetHost': 'http://recsys-proxy-sgx-1:8080', 'concurrency': 10}
db:_targetHost-concurrency_20200426223825
[{'duration': 300, 'clients': '375', 'replicaCount': 2, 'targetHost': 'http://recsys-proxy-sgx-1:8080', 'concurrency': 10, 'harnessReplicas': 9, 'timeout': 1200, 'asyncFluentd': 'false', 'timeoutRequest': 800, 'periodRequest': 1000, 'enableEncryption': '1', 'randomShift': '1', 'name': 'macro2d_375_0', 'harness': 'true'}]
helm delete --purge stub
1
helm delete --purge injector
0
helm delete --purge recsys
0
helm init --client-only
kubectl apply -f ..
helm install --name recsys --namespace default . --set hostAddress=10-0-0-33.nip.io --set mongo.replicaSet.replicas.secondary=0 --set elastic.replicas=3 --set harness.replicaCount=1 --set proxy-sgx.replicaCount=3 --set proxy-sgx.proxyType1=1 --set proxy-sgx.proxyType2=2 --set proxy-sgx.enableEncryption=1 --set proxy-sgx.bufferShuffling=10 --set proxy-sgx.debug=0 --set mongo

## macro 2d 750 2i

In [ ]:
remove_working_db()
for replicas in tqdm_notebook([9], "replicas"):
    for client in tqdm_notebook(["375"], "input rps"):
        for identifier in tqdm_notebook(range(6), "runs per config"):
            concurrency = 10

            name = "macro2d_{}_{}".format(client, identifier)
            params_recsys = {
                "hostAddress":hostAddress,
                "mongo!replicaSet!replicas!secondary":0,
                "elastic!replicas":3,
                "harness!replicaCount":1,
                "proxy-sgx!replicaCount":3,
                "proxy-sgx!proxyType1":"1",
                "proxy-sgx!proxyType2":"2",
                "proxy-sgx!enableEncryption":"1",
                "proxy-sgx!bufferShuffling":"10", 
                "proxy-sgx!debug":"0",                    
                "mongo!replicaSet!enabled": "false",
                "elastic!esJavaOpts":'"-Xmx16g -Xms16g"',
                "elastic!resources!requests!cpu":"1500m",
                "elastic!resources!requests!memory":"17Gi",
                "elastic!resources!limits!cpu":"2000m",
                "elastic!resources!limits!memory":"17Gi",
                "name":name,
                "timeout":450,
            #    "mongo!clusterDomain":"nuc.local"  # Louvain
                "mongo!clusterDomain":"cluster.local" # Bordeaux
            }
            params_injector={
                "duration":300,
                "clients":client,
                "replicaCount":2,
                "targetHost":["http://recsys-proxy-sgx-1:8080"], # pprox                
                "concurrency":[concurrency],
                "harnessReplicas": replicas,
                "timeout":1200,
                "asyncFluentd":"false",
                "timeoutRequest":800,
                "periodRequest":1000,
                "enableEncryption":"1",
                "randomShift":"1",
                "name":name,
                "harness":"true" # false if stub
            }
            params = {
                "start_time":time.time(),
                "params_recsys":params_recsys,
                "params_injector":params_injector,
                "name:":name,
                "error_module":"",
                "error":0,
                "status":"created",
                "name":name,
                "stub":"0"
            }

            loop_run_test(params)
fusion_experiments_db("private-recsys", "macro2d", False)   

In [7]:
fusion_experiments_db("private-recsys", "macro2d", False)   

private-recsys
